In [2]:
import os
import kagglehub
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

C:\Users\admin\miniconda3\envs\gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

path = kagglehub.dataset_download("shaunthesheep/microsoft-catsvsdogs-dataset")
print("Path to dataset files:", path)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 788M/788M [05:11<00:00, 2.65MB/s]

Extracting files...


Path to dataset files: C:\Users\admin\.cache\kagglehub\datasets\shaunthesheep\microsoft-catsvsdogs-dataset\versions\1


In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Изменяем размер изображений
    transforms.ToTensor(),          # Преобразуем изображения в тензоры
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Нормализация
])

# Загрузка тренировочного и тестового наборов данных
train_dataset = datasets.ImageFolder(root=f"{dataset_path}/train", transform=transform)
test_dataset = datasets.ImageFolder(root=f"{dataset_path}/test", transform=transform)

# Создаем DataLoader для батчирования данных
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

NameError: name 'transforms' is not defined

In [43]:
# Step 1: Data Preparation
def load_and_clean_data(base_dir):
    categories = ['Cat', 'Dog']
    images = []
    labels = []

    for label, category in enumerate(categories):
        category_path = os.path.join(base_dir, category)
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            try:
                with Image.open(img_path) as img:
                    img.verify()  # Verify that image is not broken
                images.append(img_path)
                labels.append(label)
            except (IOError, SyntaxError):
                print(f"Skipping broken file: {img_path}")

    return np.array(images), np.array(labels)

In [44]:
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 30
DATA_DIR = path  # Replace with your dataset directory

# Load and clean data
image_paths, labels = load_and_clean_data(DATA_DIR)

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42)

In [45]:
# Data Generators
def preprocess_image(img_path, label):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)  # Ensure 3 channels for all images
    img = tf.cond(
        tf.shape(img)[-1] != 3,
        lambda: tf.image.grayscale_to_rgb(img),  # Convert grayscale to RGB if needed
        lambda: img
    )
    img = tf.image.resize(img, IMG_SIZE)
    img = img / 255.0  # Normalize pixel values
    return img, label

In [46]:
train_data = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_data = train_data.map(preprocess_image).shuffle(len(train_paths)).batch(BATCH_SIZE)

test_data = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_data = test_data.map(preprocess_image).batch(BATCH_SIZE)

ValueError: in user code:

    File "C:\Users\admin\AppData\Local\Temp\ipykernel_18808\1904838103.py", line 5, in preprocess_image  *
        img = tf.cond(

    ValueError: Dimension 2 in both shapes must be equal, but are 9 and 3. Shapes are [?,?,9] and [?,?,3].


In [ ]:
# Step 2: Model Construction
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Step 3: Model Training
history = model.fit(train_data, validation_data=test_data, epochs=EPOCHS)

In [ ]:
# Step 4: Visualization
def plot_metrics(history):
    plt.figure(figsize=(12, 4))

    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

plot_metrics(history)

In [ ]:
# Step 5: Prediction on a sample image
sample_img_path = test_paths[0]
sample_label = test_labels[0]
sample_img, _ = preprocess_image(sample_img_path, sample_label)
sample_img = tf.expand_dims(sample_img, axis=0)  # Add batch dimension
prediction = model.predict(sample_img)[0][0]

plt.imshow(tf.squeeze(sample_img), cmap='gray')
plt.title(f'Predicted: {"Dog" if prediction > 0.5 else "Cat"'}, True Label: {"Dog" if sample_label == 1 else "Cat"'})
plt.axis('off')
plt.show()